In [ ]:
import os
import numpy as np
import pandas
import time
import random
import matplotlib
import sklearn.linear_model
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.stats
import matplotlib.offsetbox as offsetbox
from matplotlib.ticker import StrMethodFormatter
import imageio
import PIL
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from itertools import product
from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
data_new = pandas.read_pickle("./pickeddata.pkl")
data_new

,Date for US Imports and Exports,Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Exports of Total Petroleum Products (Thousand Barrels per Day),Date for Retail Gas Price,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Season
0,9-Nov-01,10772,9639,1133,12-Nov-01,1.224,autumn
1,16-Nov-01,10243,8879,1364,19-Nov-01,1.208,autumn
2,23-Nov-01,9576,8187,1389,26-Nov-01,1.168,autumn
3,30-Nov-01,11170,9856,1314,3-Dec-01,1.149,autumn
4,7-Dec-01,9885,8966,919,10-Dec-01,1.136,winter
...,...,...,...,...,...,...,...
1137,25-Aug-23,-1684,2089,-3773,28-Aug-23,3.931,summer
1138,1-Sep-23,-2593,1838,-4432,4-Sep-23,3.925,autumn
1139,8-Sep-23,431,4492,-4061,11-Sep-23,3.941,autumn
1140,15-Sep-23,-2290,1450,-3741,18-Sep-23,4.001,autumn


In [ ]:
season = {'spring': 0, 'summer': 1, 'autumn':2, 'winter': 3}
data_new.Season = [season[i] for i in data_new.Season]
data_new

,Date for US Imports and Exports,Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Exports of Total Petroleum Products (Thousand Barrels per Day),Date for Retail Gas Price,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Season
0,9-Nov-01,10772,9639,1133,12-Nov-01,1.224,2
1,16-Nov-01,10243,8879,1364,19-Nov-01,1.208,2
2,23-Nov-01,9576,8187,1389,26-Nov-01,1.168,2
3,30-Nov-01,11170,9856,1314,3-Dec-01,1.149,2
4,7-Dec-01,9885,8966,919,10-Dec-01,1.136,3
...,...,...,...,...,...,...,...
1137,25-Aug-23,-1684,2089,-3773,28-Aug-23,3.931,1
1138,1-Sep-23,-2593,1838,-4432,4-Sep-23,3.925,2
1139,8-Sep-23,431,4492,-4061,11-Sep-23,3.941,2
1140,15-Sep-23,-2290,1450,-3741,18-Sep-23,4.001,2


In [ ]:
XY=data_new[['Date for US Imports and Exports','Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day)','Season','Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)']].dropna(axis='index')
(feature1,featurename1)=('Date for US Imports and Exports',"Year")
(feature2,featurescale,featurename2)=('Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day)',1000,"Exports")
(feature3,featurename3)=('Season','Season')
(label,labelname)=('Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)',"Prices (Dollars per Gallon)")


XY.columns=[featurename1,featurename2,featurename3,labelname]
XY[featurename1]=pandas.to_datetime(XY[featurename1]).dt.year

print(XY)

#X=XY[[featurename1,featurename2,featurename3]].squeeze()
#Y=XY[labelname].squeeze()
#print(X)
#print(Y)

      Year  Exports  Season  Prices (Dollars per Gallon)
0     2001    10772       2                        1.224
1     2001    10243       2                        1.208
2     2001     9576       2                        1.168
3     2001    11170       2                        1.149
4     2001     9885       3                        1.136
...    ...      ...     ...                          ...
1137  2023    -1684       1                        3.931
1138  2023    -2593       2                        3.925
1139  2023      431       2                        3.941
1140  2023    -2290       2                        4.001
1141  2023    -1706       2                        3.963

[1142 rows x 4 columns]


In [ ]:
#XY.to_csv('output_file.csv', index=False)
df_read = pandas.read_csv('output_file.csv')

In [ ]:
df_read

,Year,Exports,Season,Prices (Dollars per Gallon)
0,2001,10772,2,1.224
1,2001,10243,2,1.208
2,2001,9576,2,1.168
3,2001,11170,2,1.149
4,2001,9885,3,1.136
...,...,...,...,...
1137,2023,-1684,1,3.931
1138,2023,-2593,2,3.925
1139,2023,431,2,3.941
1140,2023,-2290,2,4.001


In [ ]:
X = X.values
Y = Y.values

In [ ]:
# Split the data
x_train, x_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)    # change traning, testing, validating parameter
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
#for mini-batch training
class Build_Data(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.len = self.x.shape[0]
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

dataset = Build_Data(x_train_tensor,y_train_tensor)

In [ ]:
class pricePredictor_4layer(torch.nn.Module):
    def __init__(self,l1,l2,l3):
        super(pricePredictor_4layer, self).__init__()
        self.fc1 = torch.nn.Linear(3, l1)
        self.fc2 = torch.nn.Linear(l1, l2)
        self.fc3 = torch.nn.Linear(l2, l3)
        self.fc4 = torch.nn.Linear(l3, 1)
        #self.dropout=torch.nn.Dropout(p=0.2)
        #self.batchnorm1=torch.nn.BatchNorm1d(l1)
        #self.batchnorm2=torch.nn.BatchNorm1d(l2)
    def forward(self, x):
        x = self.fc1(x)
        #x=self.batchnorm1(x)
        x=torch.relu(x)
        #x=self.dropout(x)
        x = self.fc2(x)
        #x=self.batchnorm2(x)
        x=torch.relu(x)
        #x=self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
def main_model_training(param_m,lr_val,layer,batch_s,epoch,optimizer_select):
    model=None
    train_loader=None
    optimizer=None
    if layer==4:
      model=pricePredictor_4layer(param_m[0],param_m[1],param_m[2])
    #else:
     # model=pricePredictor_3layer(param_m[0],param_m[1])
    Loss = torch.nn.MSELoss()

    #if batch_s==float('inf'):
      #train_loader = DataLoader(dataset=dataset)
    train_loader = DataLoader(dataset=dataset, batch_size=batch_s)
    if optimizer_select=='Adam':
      optimizer = torch.optim.Adam(model.parameters(), lr=lr_val)
    elif optimizer_select=='SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=lr_val)
    elif optimizer_select=='RMSprop':
      optimizer = torch.optim.RMSprop(model.parameters(), lr=lr_val)

    train_losses = []
    val_losses = []
    device = "cpu"
    #if torch.cuda.is_available():
     #   device = "cuda:0"
    model.to(device)
    num_epochs = epoch
    for epoch in range(num_epochs):
      for x, y in train_loader:
        # Train
        #print('x',x)
        #print('y',y)
        model.train()
        optimizer.zero_grad()
        outputs = model(x)
        loss = Loss(outputs, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

      # Validate
      model.eval()
      val_outputs = model(x_val_tensor)
      val_loss = Loss(val_outputs, y_val_tensor)
      val_losses.append(val_loss.item())
      if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    return model,train_losses,val_losses

In [ ]:
def train_model_layer(param_m,lr_val,layer,batch_s,epoch_num,optimizer_select):
    dict_param={}
    for i in range(len(param_m)):
        save_loss={}
        dict_loss={}
        for j in range(len(lr_val)):
            _,train_losses,val_losses=main_model_training(param_m[i],lr_val[j],layer,batch_s,epoch_num,optimizer_select)
            save_loss[j]=val_losses[-1]
        min_key = min(save_loss, key=save_loss.get)
        min_value = save_loss[min_key]
        dict_loss[min_key]=min_value
        dict_param[i]=dict_loss
    return dict_param

The best model is

best_param=[8,8,32] \
best_no_layer=4 \
with Adam optimizer


In [ ]:
best_param=[8,8,32]
best_lr=0.001
best_no_layer=4

In [ ]:
model_final,train_losses,val_losses=main_model_training(best_param,best_lr,best_no_layer,128,5000,'Adam')

Epoch 1/5000, Training Loss: 71.1513900756836, Validation Loss: 50.70330810546875
Epoch 11/5000, Training Loss: 1.7456027269363403, Validation Loss: 0.7760196924209595
Epoch 21/5000, Training Loss: 1.3223077058792114, Validation Loss: 0.6683669090270996
Epoch 31/5000, Training Loss: 0.8942738175392151, Validation Loss: 0.5920575857162476
Epoch 41/5000, Training Loss: 0.5691721439361572, Validation Loss: 0.5463066697120667
Epoch 51/5000, Training Loss: 0.46237701177597046, Validation Loss: 0.4853442907333374
Epoch 61/5000, Training Loss: 0.45277851819992065, Validation Loss: 0.48017632961273193
Epoch 71/5000, Training Loss: 0.43852919340133667, Validation Loss: 0.479720801115036
Epoch 81/5000, Training Loss: 0.42125922441482544, Validation Loss: 0.48101794719696045
Epoch 91/5000, Training Loss: 0.40775617957115173, Validation Loss: 0.4814207851886749
Epoch 101/5000, Training Loss: 0.3987495005130768, Validation Loss: 0.4850156903266907
Epoch 111/5000, Training Loss: 0.3951099216938019, 

**Feature Importance**

The permutation approach to feature importance evaluation may exhibit limitations, because the performance metrics calculated by permuting individual features do not provide meaningful insights into the actual importance of each feature. This is because the order of significance tends to fluctuate across iterations as also shown in the baseline model.

In [ ]:
def shuffle_column(tensor, column_index):
    column_values = tensor[:, column_index]
    random_indices = torch.randperm(column_values.size(0))
    shuffled_column = column_values[random_indices]
    tensor[:, column_index] = shuffled_column
    return tensor

In [ ]:
def compare_importance(model,x_tensor,y_tensor):
  loss_compare=np.zeros((x_tensor.shape[1]))
  #print(x_tensor.shape[1])
  n=1000
  loss_each_run=np.zeros((n))
  for i in range(x_tensor.shape[1]):
    #print(i)
    for j in range(n):
      shuffled_tensor=shuffle_column(x_tensor, i)
      #print(shuffled_tensor)
      device = "cpu"
      if torch.cuda.is_available():
          device = "cuda:0"
      model.to(device)
      model.eval()
      output = model(x_tensor.to(device))
      metric = torch.nn.L1Loss()
      loss = metric(output, y_tensor.to(device))
      loss_each_run[j]=loss
    loss_compare[i]=np.average(loss_each_run)
  return loss_compare

In [ ]:
loss_compare_val=compare_importance(model_final,x_val_tensor,y_val_tensor)

In [ ]:
print('Validation Set: Metric of each feature permutaion ',loss_compare_val)

Validation Set: Metric of each feature permutaion  [0.49326371 0.69135552 0.68347301]


Feature Importance from Baseline Model

In [ ]:
baseline = load('regression_deg9.joblib')

In [ ]:
def compare_importance_baseline(base,deg,x_tensor,y_tensor):
    loss_compare=np.zeros((x_tensor.shape[1]))
    #print(x_tensor.shape[1])
    n=1000
    loss_each_run=np.zeros((n))
    for i in range(x_tensor.shape[1]):
      #print(i)
      for j in range(n):
        shuffled_tensor=shuffle_column(x_tensor, i)
        #print(shuffled_tensor)
        input=shuffled_tensor.numpy()
        poly = PolynomialFeatures(degree=deg, include_bias=True)
        poly_features_val = poly.fit_transform(input)
        pred_Y = base.predict(poly_features_val)
        #MSE=mean_squared_error(Yvalidate, pred_Y)
        MAE=mean_absolute_error(y_tensor.numpy().reshape(-1), pred_Y)
        loss_each_run[j]=MAE
      loss_compare[i]=np.average(loss_each_run)
    return loss_compare

In [ ]:
loss_baseline=compare_importance_baseline(baseline,9,x_val_tensor,y_val_tensor)

In [ ]:
print('Baseline: Metric of each feature permutaion ',loss_baseline)

Baseline: Metric of each feature permutaion  [1.63134968 1.66588419 1.69664181]
